In [ ]:
import snap
import os.path

# Global Variables

In [ ]:
__no_label__ = "NO_LABEL"

# Load datasets

In [ ]:
# Generic funtion to read edge file into a network
# TODO: 
# allow any graph type not only snap.TNEANet
# check if binary exists, if so, load that (rename function to make graph/net)
def edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr, tab_separated=False, dump=False):
    context = snap.TTableContext()

    schema = snap.Schema()
    schema.Add(snap.TStrTAttrPr("srcID", snap.atInt))
    schema.Add(snap.TStrTAttrPr("dstID", snap.atInt))
    
    edgeattrv = snap.TStrV()
    for attr, val in edge_attr.items():
        edgeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))
    
    srcnodeattrv = snap.TStrV()
    for attr, val in src_node_attr.items():
        srcnodeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))

    dstnodeattrv = snap.TStrV()
    for attr, val in dst_node_attr.items():
        dstnodeattrv.Add(attr)
        schema.Add(snap.TStrTAttrPr(attr, val))

    if tab_separated:
        separator = '\t'
    else:
        separator = ' '

    table = snap.TTable.LoadSS(schema, filename, context, separator, snap.TBool(False))

    # net will be an object of type snap.TNEANet
    net = table.ToNetwork(snap.TNEANet, "srcID", "dstID", srcnodeattrv, dstnodeattrv, edgeattrv, snap.aaFirst)

    if dump:
        net.Dump()

    # Save to binary
    outfile = filename + ".bin"
    FOut = snap.TFOut(outfile)
    table.Save(FOut)
    FOut.Flush()

    return net

def make_residence_hall_network():
    filename = "data/moreno_oz/out.moreno_oz_oz"

    edge_attr = {"relationship" : snap.atInt}
    src_node_attr = {}
    dst_node_attr = {}

    return edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr)

def make_pg_paper_network():
    filename = "data/example/pg_paper.txt"

    edge_attr = {"edgelabel" : snap.atStr}
    src_node_attr = {}
    dst_node_attr = {}

    return edge_file_to_network(filename, edge_attr, src_node_attr, dst_node_attr)

In [ ]:
residence_hall = make_residence_hall_network()
pg_paper = make_pg_paper_network()

# Grouping

In [ ]:
# The 1st attribute of an edge is the edge label
def get_edge_ids_per_label(network):
    labels = {}
    for EI in network.Edges():
        edge_id = EI.GetId()
        attr_values = snap.TStrV()
        network.AttrValueEI(edge_id, attr_values)
        label = attr_values[0]
        labels.setdefault(label, snap.TIntV()).append(edge_id)
    return labels

def compute_groupings(network):
    labels_edge_ids = get_edge_ids_per_label(network)
    groupings = {}
    node_ids_in_groupings = snap.TIntV()
    for label, edge_ids in sorted(labels_edge_ids.items(), 
                                    key=lambda item: len(item[1]), 
                                    reverse=True):
        sub_graph = network.GetESubGraph(edge_ids)
        # We cannot use DelNodes; an exception is thrown if we try to remove a 
        # node that is not there. There is not way to continue after the 
        # exception is thrown, hence not all nodes get removed.
        for node_id in node_ids_in_groupings:
            try:
                sub_graph.DelNode(node_id)
            except Exception:
                pass
        for NI in sub_graph.Nodes():
            node_ids_in_groupings.append(NI.GetId())
        if not sub_graph.Empty():
            groupings[label] = sub_graph

    # We must place nodes with degree zero in their own grouping
    in_deg_v = network.GetNodeInDegV()
    out_deg_v = network.GetNodeOutDegV()
    zero_deg_nodes = snap.TIntV()
    for node_id_in_deg in in_deg_v:
        if node_id_in_deg.GetVal2() == 0:
            zero_deg_nodes.Add(node_id_in_deg.GetVal1())
    for node_id_in_deg in out_deg_v:
        if node_id_in_deg.GetVal2() != 0:
            zero_deg_nodes.DelIfIn(node_id_in_deg.GetVal1())
    if not zero_deg_nodes.Empty():
        sub_graph = network.GetSubGraph(zero_deg_nodes)
        groupings[__no_label__] = sub_graph

    return groupings

# Evaluation

In [ ]:
class SuperNode:
    def __init__(self, grouping, id, sub_graph):
        self.grouping = grouping
        self.id = id
        self.sub_graph = sub_graph

def evaluation(network, groupings):
    super_nodes = {}
    super_node_id = 1
    for label, sub_graph in groupings.items():
        print("Size of %s grouping: %d" % (label, sub_graph.GetNodes()))
        components = sub_graph.GetWccs()
        for component in components:
            print("Size of subgrouping: %d" % component.Len())
            # Need to turn the component into a list because GetSubGraph only 
            # takes TIntV and component is of type TCnCom. According to the 
            # docs it should be of type TIntV (it essentially is), but I don't 
            # know how to get python to see that.
            component_as_list = list(component)
            wcc_sub_graph = network.GetSubGraph(component_as_list)
            super_node = SuperNode(label, super_node_id, wcc_sub_graph)
            super_nodes[super_node_id] = super_node
            super_node_id = super_node_id + 1
    # for id, s_node in super_nodes.items():
    #     print("Id: %d; Size: %d" %(id, s_node.sub_graph.GetNodes()))
    return super_nodes

In [ ]:
pg_paper_groupings = compute_groupings(pg_paper)

In [ ]:
print(pg_paper.GetNodes())
pg_paper_groupings = compute_groupings(pg_paper)
super_nodes = evaluation(pg_paper, pg_paper_groupings)
print(super_nodes[1].sub_graph.GetNodes())
super_nodes[1].sub_graph.Dump()

In [ ]:
print(residence_hall.GetNodes())
residence_hall_groupings = compute_groupings(residence_hall)